# 3 eme Version - Amélioration du cadre d'analyse

In [14]:
import numpy as np
import random 
import simpy
import time
from datetime import datetime

L'objectif est d'améliorer le cadre de simulation de la version précédente en y implémentant un niveau d'aléatoiriété supplémentaire. En effet il faut implémenter le fait que le nombre de panneaux pouvant proposer des bid requests est variable et évolue dynamiquement. Ainsi, le délai entre chaque bid request ne suit plus seulement une loi à paramètre constant mais à paramètre variable. L'intérêt est ainsi de simuler le fait qu'à certaines heures de la journée le délai entre deux br est beaucoup plus court que d'autres heures.
    
Le second objectif est de pouvoir simuler ce qu'il s'est passé sur un jour, puis une semaine puis un mois par exemple et stocker les résultats dans un fichier (csv ou json par exemple) afin de réutiliser ces résultats dans la mise en place d'un algorithme de pacing plus élaboré que précédemment.

### Etape 1 - Générer les fonctions des variables aléatoires ainsi que les constantes

In [15]:
# Constantes globales
nombre_jours = 1 
nombre_panneaux = 10000

In [16]:
def panneaux_dispo(nb_heures, total_panneaux):
    prop = np.random.normal(loc=0.8, scale=0.2, size=nb_heures)
    pann_dispo = list(map(lambda x: int(total_panneaux*x if x<=1 else total_panneaux), prop))
    return pann_dispo

La difficulté majeure pour cette simulation est de trouver une bonne approche afin de simuler le fait que le $\lambda$  de la loi de Poisson soit lui même une variable aléatoire (qui pourrait varier chaque heure par exemple) et donc de faire le lien entre cette variable aléatoire et le temps entre chaque bid request (plus il y a de panneaux dispo sur une heure donnée et plus le délai entre deux br est court)

In [17]:
def lambda_br(pann_dispo):
    lam = list(map(lambda x: int((1000-x)/50) if x<950 else 1, pann_dispo))
    return lam

In [52]:
def delai(lam):
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (2% de chance d'en avoir un)
    #if not random.random() < 0.98:
     #   secondes = np.random.poisson(lam + 1000)
    return secondes

In [48]:
def imps():
    lam = int(np.random.normal(loc=4, scale=2, size=1))
    if lam < 1:
        lam = 1
    nb_imp = np.random.poisson(lam)
    return nb_imp

In [59]:
def open_rtb(env, P, timestampnow):
    dispo = panneaux_dispo(10, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').minute
    i = 0
    while True:
        rt = (timestampnow+600)-env.now
        print(f"Remaining {rt} seconds")
        #Timestamp de la br
        time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
        print(f"Bid request at {time}")
        
        # Nombre d'impressions
        nb_imp = imps()
        print(f"{nb_imp} impressions")
        Prix = P * nb_imp
        print(f"Price of {Prix}€")
        
        # Detecter quelle heure est-t-il
        if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').minute != current_hour:
            i += 1 
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').minute
        
        # Temps avant la prochaine BR
        # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
        try:
            print(f"Lambda égal a {lam_secondes[i]}\n")
            time_before_next = delai(lam_secondes[i])
        except IndexError:
            print(f"Lambda égal a {lam_secondes[i-1]}\n")
            time_before_next = delai(lam_secondes[i-1])
            
        # Fin de la simulation
        if rt < time_before_next:
            print("\n Fin de la simulation !")
            
        
        yield env.timeout(time_before_next)

In [60]:
timestampnow = int(time.time())
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, 1, timestampnow))

In [62]:
env.run(until=timestampnow + 600)

ValueError: until(=1590505977) must be > the current simulation time.

In [80]:
def delai(lam):
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (1% de chance d'en avoir un)
    if not random.random() < 0.99:
        secondes = np.random.poisson(lam + 1000)
    return secondes

In [86]:
def open_rtb(env, P, timestampnow, nb_heures):
    dispo = panneaux_dispo(nb_heures, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    i = 0
    identifiant = 0
    while True:
        identifiant += 1
        Identifiants.append(identifiant)
        rt = (timestampnow+nb_heures*3600)-env.now
        
        #Timestamp de la br
        time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
        Timestampstring.append(time)
        Timestamps.append(env.now)
        
        # Nombre d'impressions
        nb_imp = imps()
        Impressions.append(nb_imp)
        Prix = P * nb_imp
        Prices.append(Prix)
        
        # Detecter quelle heure est-t-il
        if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
            i += 1 
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
        
        # Temps avant la prochaine BR
        # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
        try:
            time_before_next = delai(lam_secondes[i])
        except IndexError:
            time_before_next = delai(lam_secondes[i-1])
            
        # Fin de la simulation
        if rt < time_before_next:
            print(f"Fin de la simulation à {time} !")
        
        yield env.timeout(time_before_next)

In [87]:
timestampnow = int(time.time())
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, 1, timestampnow, 5))

In [88]:
Identifiants = []
Timestampstring = []
Timestamps = []
Impressions = []
Prices = []
env.run(until=timestampnow + 5*3600)

Fin de la simulation à 05-26-2020 22:30:17 !


In [89]:
len(Identifiants)

1057